In [1]:
import sys
import os
import joblib
import pandas as pd
import torch

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')
sys.path.insert(0, '../../../../../..')

from reimplemented_approaches.proactive_conformance_checking.data_prep_split_encode import DeviationLabeling, TrainTestSplit, Undersampling, PrefixDataset

In [2]:
# Data loading:
name_event_log = "BPIC20"
path_event_log = "../../../../../../../../../data/data/DomesticDeclarations.csv"
path_process_model = "../../../../../../../data/process_models/BPIC20-DD.bpmn"

In [3]:
# Define attributes:
# Load the event log as pandas dataframe. Get all attributes in the log:
df = pd.read_csv(path_event_log)
print(df.columns)

Index(['id', 'org:resource', 'concept:name', 'time:timestamp', 'org:role',
       'case:id', 'case:concept:name', 'case:BudgetNumber',
       'case:DeclarationNumber', 'case:Amount'],
      dtype='object')


In [4]:
df

,id,org:resource,concept:name,time:timestamp,org:role,case:id,case:concept:name,case:BudgetNumber,case:DeclarationNumber,case:Amount
0,st_step 86794_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 09:49:50.000000,EMPLOYEE,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205
1,st_step 86793_0,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2017-01-09 11:27:48.000000,SUPERVISOR,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205
2,dd_declaration 86791_19,SYSTEM,Request Payment,2017-01-10 09:34:44.000000,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205
3,dd_declaration 86791_20,SYSTEM,Payment Handled,2017-01-12 17:31:22.000000,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205
4,st_step 86798_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 10:26:14.000000,EMPLOYEE,declaration 86795,declaration 86795,budget 86566,declaration number 86796,182.464172
...,...,...,...,...,...,...,...,...,...,...
56432,st_step 138363_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-12-29 17:50:14.000000,EMPLOYEE,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576
56433,st_step 138361_0,STAFF MEMBER,Declaration APPROVED by ADMINISTRATION,2018-12-29 17:56:13.000000,ADMINISTRATION,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576
56434,st_step 138362_0,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2019-01-03 08:55:52.000000,SUPERVISOR,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576
56435,dd_declaration 138359_19,SYSTEM,Request Payment,2019-01-08 08:20:28.000000,UNDEFINED,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576


In [5]:
# Data preparation and labelling
dl = DeviationLabeling(log_name=name_event_log,
                       path_event_log=path_event_log,
                       path_process_model=path_process_model,
                       label_strategy='collective')

# collective
df_labeled_deviations, encoders = dl.generate_individual_labels(trace_attr=['case:Amount'], conf_runs=1)

  0%|          | 0/1 [00:00<?, ?it/s]

aligning log, completed variants ::   0%|          | 0/99 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  2.26it/s]


In [6]:
df_labeled_deviations

,case_id,activities,resources,months,trace_attr_case:Amount,"y_('>>', 'Declaration APPROVED by ADMINISTRATION')","y_('>>', 'Declaration FINAL_APPROVED by SUPERVISOR')","y_('>>', 'Declaration REJECTED by ADMINISTRATION')","y_('>>', 'Declaration REJECTED by EMPLOYEE')","y_('>>', 'Declaration SUBMITTED by EMPLOYEE')",...,"y_('Declaration FOR_APPROVAL by ADMINISTRATION', '>>')","y_('Declaration FOR_APPROVAL by PRE_APPROVER', '>>')","y_('Declaration FOR_APPROVAL by SUPERVISOR', '>>')","y_('Declaration REJECTED by EMPLOYEE', '>>')","y_('Declaration REJECTED by MISSING', '>>')","y_('Declaration REJECTED by PRE_APPROVER', '>>')","y_('Declaration SAVED by EMPLOYEE', '>>')","y_('Declaration SUBMITTED by EMPLOYEE', '>>')","y_('Payment Handled', '>>')","y_('Request Payment', '>>')"
0,"('declaration 100000', 1)","[15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.138922,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"('declaration 100000', 2)","[15, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1.138922,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"('declaration 100000', 3)","[15, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1.138922,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"('declaration 100000', 4)","[15, 1, 4, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.138922,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"('declaration 100000', 5)","[15, 1, 4, 17, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1.138922,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56432,"('declaration 99995', 1)","[15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-0.355713,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56433,"('declaration 99995', 2)","[15, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-0.355713,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56434,"('declaration 99995', 3)","[15, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-0.355713,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56435,"('declaration 99995', 4)","[15, 1, 4, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-0.355713,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
act_ids = encoders['activity_ids']
print("Activity ids: " ,act_ids)
print("\n")

res_ids = encoders['resource_ids']
print("Reource ids: ", res_ids)
print("\n")

months = encoders['month_ids']
print("Months: ", months)
print("\n")

deviations = encoders['deviations']
print("Deviations: ", deviations)
print("\n")

Activity ids:  {'Declaration APPROVED by ADMINISTRATION': 1, 'Declaration APPROVED by BUDGET OWNER': 2, 'Declaration APPROVED by PRE_APPROVER': 3, 'Declaration FINAL_APPROVED by SUPERVISOR': 4, 'Declaration FOR_APPROVAL by ADMINISTRATION': 5, 'Declaration FOR_APPROVAL by PRE_APPROVER': 6, 'Declaration FOR_APPROVAL by SUPERVISOR': 7, 'Declaration REJECTED by ADMINISTRATION': 8, 'Declaration REJECTED by BUDGET OWNER': 9, 'Declaration REJECTED by EMPLOYEE': 10, 'Declaration REJECTED by MISSING': 11, 'Declaration REJECTED by PRE_APPROVER': 12, 'Declaration REJECTED by SUPERVISOR': 13, 'Declaration SAVED by EMPLOYEE': 14, 'Declaration SUBMITTED by EMPLOYEE': 15, 'Payment Handled': 16, 'Request Payment': 17}


Reource ids:  {'STAFF MEMBER': 1, 'SYSTEM': 2}


Months:  {'10_2017': 1, '10_2018': 2, '11_2017': 3, '11_2018': 4, '12_2017': 5, '12_2018': 6, '1_2017': 7, '1_2018': 8, '1_2019': 9, '2_2017': 10, '2_2018': 11, '2_2019': 12, '3_2017': 13, '3_2018': 14, '3_2019': 15, '4_2017': 16, '4_201

In [8]:
# Save prepared dataframe as .csv
os.makedirs(".", exist_ok=True)
csv_path_collective = os.path.join("./prefix_deviations.csv")
df_labeled_deviations.to_csv(csv_path_collective, index=False)

In [9]:
# Train and test split
tts = TrainTestSplit(df_labled_deviations=df_labeled_deviations,
                     label_strategy = "collective")

train_df, val_df, test_df = tts.data_split(val_frac=0.2)

In [10]:
train_df

,case_id,activities,resources,months,trace_attr_case:Amount,"y_('>>', 'Declaration APPROVED by ADMINISTRATION')","y_('>>', 'Declaration FINAL_APPROVED by SUPERVISOR')","y_('>>', 'Declaration REJECTED by ADMINISTRATION')","y_('>>', 'Declaration REJECTED by EMPLOYEE')","y_('>>', 'Declaration SUBMITTED by EMPLOYEE')",...,"y_('Declaration FOR_APPROVAL by ADMINISTRATION', '>>')","y_('Declaration FOR_APPROVAL by PRE_APPROVER', '>>')","y_('Declaration FOR_APPROVAL by SUPERVISOR', '>>')","y_('Declaration REJECTED by EMPLOYEE', '>>')","y_('Declaration REJECTED by MISSING', '>>')","y_('Declaration REJECTED by PRE_APPROVER', '>>')","y_('Declaration SAVED by EMPLOYEE', '>>')","y_('Declaration SUBMITTED by EMPLOYEE', '>>')","y_('Payment Handled', '>>')","y_('Request Payment', '>>')"
0,"('declaration 100000', 2)","[15, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1.138922,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"('declaration 100000', 5)","[15, 1, 4, 17, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1.138922,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"('declaration 100005', 2)","[15, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.173612,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"('declaration 100005', 3)","[15, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.173612,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"('declaration 100005', 4)","[15, 1, 4, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.173612,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30095,"('declaration 99983', 4)","[15, 1, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 8, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1.173310,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30096,"('declaration 99989', 1)","[15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.264191,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30097,"('declaration 99989', 2)","[15, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.264191,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30098,"('declaration 99989', 3)","[15, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.264191,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Undersampling train set
u = Undersampling(train_data=train_df, 
                  list_dynamic_cols=['activities', 'resources', 'months'],
                  label_strategy='collective')

train_df, y_no_true_class = u.one_sided_selection_undersampling()

In [12]:
train_df

,case_id,activities,resources,months,trace_attr_case:Amount,"y_('>>', 'Declaration APPROVED by ADMINISTRATION')","y_('>>', 'Declaration REJECTED by ADMINISTRATION')","y_('>>', 'Declaration REJECTED by EMPLOYEE')","y_('>>', 'Declaration SUBMITTED by EMPLOYEE')","y_('>>', 'Payment Handled')",...,"y_('Declaration APPROVED by PRE_APPROVER', '>>')","y_('Declaration FINAL_APPROVED by SUPERVISOR', '>>')","y_('Declaration FOR_APPROVAL by ADMINISTRATION', '>>')","y_('Declaration REJECTED by EMPLOYEE', '>>')","y_('Declaration REJECTED by MISSING', '>>')","y_('Declaration REJECTED by PRE_APPROVER', '>>')","y_('Declaration SAVED by EMPLOYEE', '>>')","y_('Declaration SUBMITTED by EMPLOYEE', '>>')","y_('Payment Handled', '>>')","y_('Request Payment', '>>')"
0,"('declaration 100000', 2)","[15, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1.138922,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"('declaration 100000', 5)","[15, 1, 4, 17, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1.138922,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"('declaration 100005', 2)","[15, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.173612,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"('declaration 100005', 3)","[15, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.173612,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"('declaration 100005', 4)","[15, 1, 4, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.173612,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30095,"('declaration 99983', 4)","[15, 1, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 8, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1.173310,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30096,"('declaration 99989', 1)","[15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.264191,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30097,"('declaration 99989', 2)","[15, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.264191,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30098,"('declaration 99989', 3)","[15, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.264191,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Save encoder ids for all attributes in the dataframe:
y_columns = cols = [c for c in train_df.columns if c.startswith("y_")]
encoders['deviations'] = y_columns
# Save encoding/ decoding key as .pkl
joblib.dump(encoders, "encoders.pkl")

['encoders.pkl']

In [14]:
# Tensor encoding and saving — quick literal fix
device = torch.device("cpu") # store to cpu
dataset_prep = PrefixDataset(# datset
                             df_train=train_df,
                             df_val=val_df,
                             df_test=test_df,
                             # column values
                             activity_col='activities',
                             resource_col='resources',
                             month_col='months',
                             # add in front of the attribute: trace_attr_
                             trace_cols=['trace_attr_case:Amount'],
                             y_cols=encoders['deviations'],
                             label_strategy = "collective")

# Encode to tensor datsets and save files
train_set, val_set, test_set = dataset_prep.tensor_datset_encoding(device=device)

dataset_prep.save_datasets(train_dataset=train_set, val_dataset=val_set, test_dataset=test_set, save_path=".")

('./train_set.pkl', './val_set.pkl', './test_set.pkl')

In [15]:
train_set, val_set, test_set = dataset_prep.load_datasets(".")
print(train_set.tensors)
print(val_set.tensors)
print(test_set.tensors)

(tensor([[15,  1,  0,  ...,  0,  0,  0],
        [15,  1,  4,  ...,  0,  0,  0],
        [15,  1,  0,  ...,  0,  0,  0],
        ...,
        [15,  1,  0,  ...,  0,  0,  0],
        [15,  1,  2,  ...,  0,  0,  0],
        [15,  1,  2,  ...,  0,  0,  0]]), tensor([[1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0],
        ...,
        [1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([[ 8, 11,  0,  ...,  0,  0,  0],
        [ 8, 11, 11,  ...,  0,  0,  0],
        [ 8,  8,  0,  ...,  0,  0,  0],
        ...,
        [ 8,  8,  0,  ...,  0,  0,  0],
        [ 8,  8,  8,  ...,  0,  0,  0],
        [ 8,  8,  8,  ...,  0,  0,  0]]), tensor([[1.1389],
        [1.1389],
        [0.1736],
        ...,
        [1.2642],
        [1.2642],
        [1.2642]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0]